[![AnalyticsDojo](https://github.com/rpi-techfundamentals/spring2019-materials/blob/master/fig/final-logo.png?raw=1)](http://rpi.analyticsdojo.com)
<center><h1> What's Cooking  in Python</h1></center>
<center><h3><a href = 'http://rpi.analyticsdojo.com'>rpi.analyticsdojo.com</a></h3></center>

# What's Cooking in Python


This was adopted from. 
https://www.kaggle.com/manuelatadvice/whats-cooking/noname/code

In [41]:
import re
from urllib.request import urlopen

import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [42]:
nltk.download("omw-1.4")
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /Users/minor/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/minor/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
train_df = pd.read_json(urlopen('https://raw.githubusercontent.com/RPI-Analytics/MGMT6963-2015/master/data/whatscooking/whatscookingtrain.json'))
test_df = pd.read_json(urlopen('https://raw.githubusercontent.com/RPI-Analytics/MGMT6963-2015/master/data/whatscooking/whatscookingtest.json'))

In [44]:
train_df

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
...,...,...,...
39769,29109,irish,"[light brown sugar, granulated sugar, butter, ..."
39770,11462,italian,"[KRAFT Zesty Italian Dressing, purple onion, b..."
39771,2238,irish,"[eggs, citrus fruit, raisins, sourdough starte..."
39772,41882,chinese,"[boneless chicken skinless thigh, minced garli..."


In [45]:
test_df

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."
...,...,...
9939,30246,"[large egg yolks, fresh lemon juice, sugar, bo..."
9940,36028,"[hot sauce, butter, sweet potatoes, adobo sauc..."
9941,22339,"[black pepper, salt, parmigiano reggiano chees..."
9942,42525,"[cheddar cheese, cayenne, paprika, plum tomato..."


In [46]:
#First we want to see the most popular cuisine for the naive model. 
train_df.groupby('cuisine').size()

cuisine
brazilian        467
british          804
cajun_creole    1546
chinese         2673
filipino         755
french          2646
greek           1175
indian          3003
irish            667
italian         7838
jamaican         526
japanese        1423
korean           830
mexican         6438
moroccan         821
russian          489
southern_us     4320
spanish          989
thai            1539
vietnamese       825
dtype: int64

In [47]:
#Here we write the most popular selection.  This is the baseline by which we will judge other models. 
# test_df['cuisine']='italian'

In [48]:
stemmer = WordNetLemmatizer()

#To interpret this, read from right to left.
train_df['ingredients_string1'] = [' '.join([stemmer.lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in train_df['ingredients']]
test_df['ingredients_string1'] = [' '.join([stemmer.lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in test_df['ingredients']]

In [49]:
train_df

,id,cuisine,ingredients,ingredients_string1
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuce black olives grape tomatoes ga...
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",plain flour ground pepper salt tomato ground b...
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",egg pepper salt mayonaise cooking oil green ch...
3,22213,indian,"[water, vegetable oil, wheat, salt]",water vegetable oil wheat salt
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",black pepper shallot cornflour cayenne pepper ...
...,...,...,...,...
39769,29109,irish,"[light brown sugar, granulated sugar, butter, ...",light brown sugar granulated sugar butter warm...
39770,11462,italian,"[KRAFT Zesty Italian Dressing, purple onion, b...",KRAFT Zesty Italian Dressing purple onion broc...
39771,2238,irish,"[eggs, citrus fruit, raisins, sourdough starte...",egg citrus fruit raisin sourdough starter flou...
39772,41882,chinese,"[boneless chicken skinless thigh, minced garli...",boneless chicken skinless thigh minced garlic ...


In [50]:
test_df

,id,ingredients,ingredients_string1
0,18009,"[baking powder, eggs, all-purpose flour, raisi...",baking powder egg all purpose flour raisin mil...
1,28583,"[sugar, egg yolks, corn starch, cream of tarta...",sugar egg yolks corn starch cream of tartar ba...
2,41580,"[sausage links, fennel bulb, fronds, olive oil...",sausage links fennel bulb frond olive oil cuba...
3,29752,"[meat cuts, file powder, smoked sausage, okra,...",meat cuts file powder smoked sausage okra shri...
4,35687,"[ground black pepper, salt, sausage casings, l...",ground black pepper salt sausage casings leek ...
...,...,...,...
9939,30246,"[large egg yolks, fresh lemon juice, sugar, bo...",large egg yolks fresh lemon juice sugar bourbo...
9940,36028,"[hot sauce, butter, sweet potatoes, adobo sauc...",hot sauce butter sweet potatoes adobo sauce salt
9941,22339,"[black pepper, salt, parmigiano reggiano chees...",black pepper salt parmigiano reggiano cheese r...
9942,42525,"[cheddar cheese, cayenne, paprika, plum tomato...",cheddar cheese cayenne paprika plum tomatoes g...


In [51]:
#http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
vectorizer = TfidfVectorizer(stop_words='english',
                                   ngram_range = ( 1 , 1 ), analyzer="word",
                                   max_df = .57, binary=False, token_pattern=r'\w+', sublinear_tf=False)
# Challenge: try other vectorizers and their parameters, such as CountVectorizer (= Bag of Words)
# vectorizer = TfidfVectorizer(stop_words='english')

In [52]:
#Note that this doesn't work with the #todense option.  
train_vec = vectorizer.fit_transform(train_df["ingredients_string1"])
test_vec = vectorizer.transform(test_df["ingredients_string1"])

In [53]:
train_vec

<39774x2963 sparse matrix of type '<class 'numpy.float64'>'
	with 727921 stored elements in Compressed Sparse Row format>

In [54]:
test_vec

<9944x2963 sparse matrix of type '<class 'numpy.float64'>'
	with 182346 stored elements in Compressed Sparse Row format>

## Logistic Regression and Regularization.

- Regularlization can help us with the large matrix by adding a penalty for each parameter. 
- Finding out how much regularization via grid search (search across hyperparameters.)

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

```C : float, default: 1.0
Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.```

In [56]:
from sklearn.model_selection import GridSearchCV
# Search a grid space instead of using the LogisticRegression classifier as-is
logistic_regression_classifier = GridSearchCV(LogisticRegression(), {"C": [1, 10]})
logistic_regression_classifier.fit(train_vec, train_df["cuisine"])
test_df['logistic_regression_cuisine'] = logistic_regression_classifier.predict(test_vec)

/Users/minor/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/minor/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logis

In [57]:
from sklearn.ensemble import RandomForestClassifier
random_forest_classifier = RandomForestClassifier(n_estimators = 10)
random_forest_classifier = random_forest_classifier.fit(train_vec, train_df["cuisine"])
test_df["random_forest_cuisine"] = random_forest_classifier.predict(test_vec)

In [58]:
test_df

,id,ingredients,ingredients_string1,logistic_regression_cuisine,random_forest_cuisine
0,18009,"[baking powder, eggs, all-purpose flour, raisi...",baking powder egg all purpose flour raisin mil...,british,southern_us
1,28583,"[sugar, egg yolks, corn starch, cream of tarta...",sugar egg yolks corn starch cream of tartar ba...,southern_us,southern_us
2,41580,"[sausage links, fennel bulb, fronds, olive oil...",sausage links fennel bulb frond olive oil cuba...,italian,italian
3,29752,"[meat cuts, file powder, smoked sausage, okra,...",meat cuts file powder smoked sausage okra shri...,cajun_creole,cajun_creole
4,35687,"[ground black pepper, salt, sausage casings, l...",ground black pepper salt sausage casings leek ...,italian,italian
...,...,...,...,...,...
9939,30246,"[large egg yolks, fresh lemon juice, sugar, bo...",large egg yolks fresh lemon juice sugar bourbo...,french,french
9940,36028,"[hot sauce, butter, sweet potatoes, adobo sauc...",hot sauce butter sweet potatoes adobo sauce salt,southern_us,southern_us
9941,22339,"[black pepper, salt, parmigiano reggiano chees...",black pepper salt parmigiano reggiano cheese r...,italian,italian
9942,42525,"[cheddar cheese, cayenne, paprika, plum tomato...",cheddar cheese cayenne paprika plum tomatoes g...,southern_us,southern_us
